In [18]:
import os, cv2
import librosa
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [9]:
data = np.load("../artifacts/data/MusicFeatures.npz")
spec = data['spec'] 
mel_spec = data['mel'] 
mfcc = data['mfcc'] 
zcr = data['zcr'] 
spec_c = data['cen'] 
chr = data['chroma'] 
y = data['target'] 

In [10]:
# train test split
spec_train ,spec_test , mel_spec_train ,mel_spec_test , mfcc_train ,mfcc_test , zcr_train,zcr_test, spec_c_train ,spec_c_test , chr_train ,chr_test , y_train  ,y_test  = train_test_split(spec, mel_spec, mfcc, zcr, spec_c, chr, y, test_size=0.2, random_state=42)

In [11]:
# Scaling data -> Spectrogram
max_spec = np.amax(spec)

spec_train = spec_train/max_spec
spec_test = spec_test/max_spec

spec_train = spec_train.astype(np.float32)
spec_test = spec_test.astype(np.float32)

In [15]:
N, row, col = spec_train.shape
spec_train = spec_train.reshape((N, row, col, 1))

N, row, col = spec_test.shape
spec_test = spec_test.reshape((N, row, col, 1))

In [16]:
spec_train.shape

(755, 1025, 1293, 1)

In [19]:
'''
    The original shape of MFCC is (944, 10, 1293). We first resize both the MFCC train and test data to 
    (944, 120, 600). After that, we reshape the data into (N, row, col, 1) for CNN. Then we standardize the 
    data.

'''

mfcc_train_new = np.empty((mfcc_train.shape[0], 120, 600))
mfcc_test_new = np.empty((mfcc_test.shape[0], 120, 600))

for i in range(mfcc_train.shape[0]):
    curr = mfcc_train[i]
    curr = cv2.resize(curr, (600, 120))
    mfcc_train_new[i] = curr

mfcc_train = mfcc_train_new


for i in range(mfcc_test.shape[0]):
    curr = mfcc_test[i]
    curr = cv2.resize(curr, (600, 120))
    mfcc_test_new[i] = curr

mfcc_test = mfcc_test_new


mfcc_train = mfcc_train.astype(np.float32)
mfcc_test = mfcc_test.astype(np.float32)


N, row, col = mfcc_train.shape
mfcc_train = mfcc_train.reshape((N, row, col, 1))


N, row, col = mfcc_test.shape
mfcc_test = mfcc_test.reshape((N, row, col, 1))


mean_data = np.mean(mfcc_train)
std_data = np.std(mfcc_train)


mfcc_train = (mfcc_train - mean_data)/std_data
mfcc_test = (mfcc_test - mean_data)/std_data

In [20]:
# Mel-Spectrogram
mel_spec_max = np.amax(mel_spec_train)
mel_train = mel_spec_train/np.amax(mel_spec_max)
mel_test = mel_spec_test/np.amax(mel_spec_max)

mel_train = mel_train.astype(np.float32)
mel_test = mel_test.astype(np.float32)

N, row, col = mel_train.shape
mel_train = mel_train.reshape((N, row, col, 1))

N, row, col = mel_test.shape
mel_test = mel_test.reshape((N, row, col, 1))

In [21]:
# Saving spectrogram features as .npz file
np.savez_compressed("../artifacts/data/spectrogram_features.npz", spec_train=spec_train, spec_test=spec_test, y_train=y_train, y_test=y_test)

In [22]:
# Saving mfcc features as .npz file
np.savez_compressed("../artifacts/data/mfcc_features.npz", mfcc_train=mfcc_train, mfcc_test=mfcc_test, y_train=y_train, y_test=y_test)

In [23]:
# Saving mel_spectrogram features as .npz file
np.savez_compressed("../artifacts/data/mel_spec_features.npz", mel_spec_train=mel_spec_train, mel_spec_test=mel_spec_test, y_train=y_train, y_test=y_test)